## Xiangyu Liu 001498478

## XGBoost 

In [1]:
import pandas as pd
import gc

In [2]:
aisles = pd.read_csv('aisles.csv')
products = pd.read_csv('products.csv')
order_products_train = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv')

In [3]:
#creating a datframe that will contain only prior information
op = pd.merge(orders, order_products_prior, on='order_id', how='inner')
op.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


## Creating features related to the users. i.e using user_id

In [4]:
#Total number of orders placed by each users
users = op.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [5]:
#average number of products bought by the user in each purchase.

#1. First getting the total number of products in each order.
total_prd_per_order = op.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()
total_prd_per_order.head(10)

,user_id,order_id,total_products_per_order
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6
5,1,2398795,6
6,1,2539329,5
7,1,2550362,9
8,1,3108588,6
9,1,3367565,4


In [6]:
#2. Getting the average products purchased by each user
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [7]:
#deleting the total_prd_per_order dataframe
del total_prd_per_order

In [8]:
#dow of most orders placed by each user
from scipy import stats
dow = op.groupby(by='user_id')['order_dow'].agg(lambda x: stats.mode(x)[0]).to_frame('dow_most_orders_u').reset_index()
dow.head()

,user_id,dow_most_orders_u
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [9]:
#hour of day when most orders placed by each user
from scipy import stats
hod = op.groupby(by='user_id')['order_hour_of_day'].agg(lambda x: stats.mode(x)[0]).to_frame('hod_most_orders_u').reset_index()
hod.head()

,user_id,hod_most_orders_u
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [10]:
#merging the user created features.

#1. merging avg_products with users
users = users.merge(avg_products, on='user_id', how='left')
#deleting avg_products
del avg_products
users.head()

,user_id,u_num_of_orders,u_avg_prd
0,1,10,5.900000
1,2,14,13.928571
2,3,12,7.333333
3,4,5,3.600000
4,5,4,9.250000


In [11]:
#2. merging dow with users.
users = users.merge(dow, on='user_id', how='left')
#deleting dow
del dow
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u
0,1,10,5.900000,4
1,2,14,13.928571,2
2,3,12,7.333333,0
3,4,5,3.600000,4
4,5,4,9.250000,3


In [12]:
#3. merging hod with users
#2. merging dow with users.
users = users.merge(hod, on='user_id', how='left')
#deleting dow
del hod
users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u
0,1,10,5.900000,4,7
1,2,14,13.928571,2,9
2,3,12,7.333333,0,16
3,4,5,3.600000,4,15
4,5,4,9.250000,3,18


## Creating features related to the products using product_id

In [13]:
#number of time a product was purchased.
prd = op.groupby(by='product_id')['order_id'].agg('count').to_frame('prd_count_p').reset_index()
prd.head()

,product_id,prd_count_p
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [14]:
#products reorder ratio.
reorder_p = op.groupby(by='product_id')['reordered'].agg('mean').to_frame('p_reordered_ratio').reset_index()
reorder_p.head()

,product_id,p_reordered_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [15]:
#merging the reorder_p with prd
prd = prd.merge(reorder_p, on='product_id', how='left')
#deleting reorder_p
del reorder_p
prd.head()

,product_id,prd_count_p,p_reordered_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.600000


## Creating user-product features

In [16]:
#how many times a user bought the same product.
uxp = op.groupby(by=['user_id', 'product_id'])['order_id'].agg('count').to_frame('uxp_times_bought').reset_index()
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [17]:
#reorder ratio of the user for each product.
reorder_uxp = op.groupby(by=['user_id', 'product_id'])['reordered'].agg('mean').to_frame('uxp_reordered_ratio').reset_index()
reorder_uxp.head()

,user_id,product_id,uxp_reordered_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [18]:
#merging the two dataframes into one
uxp = uxp.merge(reorder_uxp, on=['user_id', 'product_id'], how='left')
#deleting reorder_uxp
del reorder_uxp
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio
0,1,196,10,0.900000
1,1,10258,9,0.888889
2,1,10326,1,0.000000
3,1,12427,10,0.900000
4,1,13032,3,0.666667


## Merging all the features into data DF.

In [19]:
#merging users df into uxp
data = uxp.merge(users, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u
0,1,196,10,0.900000,10,5.9,4,7
1,1,10258,9,0.888889,10,5.9,4,7
2,1,10326,1,0.000000,10,5.9,4,7
3,1,12427,10,0.900000,10,5.9,4,7
4,1,13032,3,0.666667,10,5.9,4,7


In [20]:
#merging products df into data
data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio
0,1,196,10,0.900000,10,5.9,4,7,35791,0.776480
1,1,10258,9,0.888889,10,5.9,4,7,1946,0.713772
2,1,10326,1,0.000000,10,5.9,4,7,5526,0.652009
3,1,12427,10,0.900000,10,5.9,4,7,6476,0.740735
4,1,13032,3,0.666667,10,5.9,4,7,3751,0.657158


In [21]:
#deleting unwanted dfs
del [users, prd, uxp]

## Creating train and test dataset

In [22]:
#keeping only the train and test eval set from the orders dataframe.
order_future = orders.loc[((orders.eval_set == 'train') | (orders.eval_set == 'test')), ['user_id', 'eval_set', 'order_id']]
order_future.head()

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797


In [23]:
#merging the order_future with the data.
data = data.merge(order_future, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,eval_set,order_id
0,1,196,10,0.900000,10,5.9,4,7,35791,0.776480,train,1187899
1,1,10258,9,0.888889,10,5.9,4,7,1946,0.713772,train,1187899
2,1,10326,1,0.000000,10,5.9,4,7,5526,0.652009,train,1187899
3,1,12427,10,0.900000,10,5.9,4,7,6476,0.740735,train,1187899
4,1,13032,3,0.666667,10,5.9,4,7,3751,0.657158,train,1187899


In [24]:
#preparing the train df.
data_train = data[data.eval_set == 'train']
data_train.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,eval_set,order_id
0,1,196,10,0.900000,10,5.9,4,7,35791,0.776480,train,1187899
1,1,10258,9,0.888889,10,5.9,4,7,1946,0.713772,train,1187899
2,1,10326,1,0.000000,10,5.9,4,7,5526,0.652009,train,1187899
3,1,12427,10,0.900000,10,5.9,4,7,6476,0.740735,train,1187899
4,1,13032,3,0.666667,10,5.9,4,7,3751,0.657158,train,1187899


In [25]:
#merging the information from the order_proucts_train df into the data_train.
data_train = data_train.merge(order_products_train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
data_train.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,eval_set,order_id,reordered
0,1,196,10,0.900000,10,5.9,4,7,35791,0.776480,train,1187899,1.0
1,1,10258,9,0.888889,10,5.9,4,7,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.000000,10,5.9,4,7,5526,0.652009,train,1187899,NaN
3,1,12427,10,0.900000,10,5.9,4,7,6476,0.740735,train,1187899,NaN
4,1,13032,3,0.666667,10,5.9,4,7,3751,0.657158,train,1187899,1.0


In [26]:
#filling the NAN values
data_train.reordered.fillna(0, inplace=True)

In [27]:
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)


In [28]:
data_train.head(20)

uxp_times_bought  uxp_reordered_ratio  u_num_of_orders  \
user_id product_id                                                           
1       196                       10             0.900000               10   
        10258                      9             0.888889               10   
        10326                      1             0.000000               10   
        12427                     10             0.900000               10   
        13032                      3             0.666667               10   
        13176                      2             0.500000               10   
        14084                      1             0.000000               10   
        17122                      1             0.000000               10   
        25133                      8             0.875000               10   
        26088                      2             0.500000               10   
        26405                      2             0.500000               10   
        30450                      1             0.000000               10   
        35951                      1             0.000000               10   
        38928                      1             0.000000               10   
        39657                      1             0.000000               10   
        41787                      1             0.000000               10   
        46149                      3             0.666667               10   
        49235                      2             0.500000               10   
2       23                         1             0.000000               14   
        79                         1             0.000000               14   

                    u_avg_prd  dow_most_orders_u  hod_most_orders_u  \
user_id product_id                                                    
1       196          5.900000                  4                  7   
        10258        5.900000                  4                  7   
        10326        5.900000                  4                  7   
        12427        5.900000                  4                  7   
        13032        5.900000                  4                  7   
        13176        5.900000                  4                  7   
        14084        5.900000                  4                  7   
        17122        5.900000                  4                  7   
        25133        5.900000                  4                  7   
        26088        5.900000                  4                  7   
        26405        5.900000                  4                  7   
        30450        5.900000                  4                  7   
        35951        5.900000                  4                  7   
        38928        5.900000                  4                  7   
        39657        5.900000                  4                  7   
        41787        5.900000                  4                  7   
        46149        5.900000                  4                  7   
        49235        5.900000                  4                  7   
2       23          13.928571                  2                  9   
        79          13.928571                  2                  9   

                    prd_count_p  p_reordered_ratio  reordered  
user_id product_id                                             
1       196               35791           0.776480        1.0  
        10258              1946           0.713772        1.0  
        10326              5526           0.652009        0.0  
        12427              6476           0.740735        0.0  
        13032              3751           0.657158        1.0  
        13176            379450           0.832555        0.0  
        14084             15935           0.810982        0.0  
        17122             13880           0.675576        0.0  
        25133              6196           0.740155        1.0  
        26088              2523     

In [29]:
#preparing the test dataset.
data_test = data[data.eval_set == 'test']
data_test.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,eval_set,order_id
120,3,248,1,0.000000,12,7.333333,0,16,6371,0.400251,test,2774568
121,3,1005,1,0.000000,12,7.333333,0,16,463,0.440605,test,2774568
122,3,1819,3,0.666667,12,7.333333,0,16,2424,0.492162,test,2774568
123,3,7503,1,0.000000,12,7.333333,0,16,12474,0.553551,test,2774568
124,3,8021,1,0.000000,12,7.333333,0,16,27864,0.591157,test,2774568


In [30]:
#deleting unwanted columns
data_test.drop(['eval_set', 'order_id'], axis=1, inplace=True)

/Library/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [31]:
#setting user_id and product_id as index.
data_test = data_test.set_index(['user_id', 'product_id'])

In [32]:
data_test.head()

uxp_times_bought  uxp_reordered_ratio  u_num_of_orders  \
user_id product_id                                                           
3       248                        1             0.000000               12   
        1005                       1             0.000000               12   
        1819                       3             0.666667               12   
        7503                       1             0.000000               12   
        8021                       1             0.000000               12   

                    u_avg_prd  dow_most_orders_u  hod_most_orders_u  \
user_id product_id                                                    
3       248          7.333333                  0                 16   
        1005         7.333333                  0                 16   
        1819         7.333333                  0                 16   
        7503         7.333333                  0                 16   
        8021         7.333333                  0                 16   

                    prd_count_p  p_reordered_ratio  
user_id product_id                                  
3       248                6371           0.400251  
        1005                463           0.440605  
        1819               2424           0.492162  
        7503              12474           0.553551  
        8021              27864           0.591157

In [33]:
#deleting unwanted df
del [aisles, departments, order_products_prior, order_products_train, orders, order_future, data] #order_future, data

In [34]:
#resetting index
data_train.reset_index(inplace=True)
data_test.reset_index(inplace=True)

In [35]:
data_train.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,hod_most_orders_u,prd_count_p,p_reordered_ratio,reordered
0,1,196,10,0.900000,10,5.9,4,7,35791,0.776480,1.0
1,1,10258,9,0.888889,10,5.9,4,7,1946,0.713772,1.0
2,1,10326,1,0.000000,10,5.9,4,7,5526,0.652009,0.0
3,1,12427,10,0.900000,10,5.9,4,7,6476,0.740735,0.0
4,1,13032,3,0.666667,10,5.9,4,7,3751,0.657158,1.0


In [36]:
#merging the aisles and department ids to with the train and test data
data_train = data_train.merge(products[['product_id', 'aisle_id']], on='product_id', how='left')
data_test = data_test.merge(products[['product_id', 'aisle_id']], on='product_id', how='left')

In [37]:
#department
data_train = data_train.merge(products[['product_id', 'department_id']], on='product_id', how='left')
data_test = data_test.merge(products[['product_id', 'department_id']], on='product_id', how='left')

In [38]:
#setting user_id and product_id as index.
data_test = data_test.set_index(['user_id', 'product_id'])
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])


## Creating Predictive model --- XGBoost

In [39]:
#Building a XGBoost model.
#importing the package.
import xgboost as xgb

#splitting the train data into training and testing set.
X_train, y_train = data_train.drop('reordered', axis=1), data_train.reordered

#setting boosters parameters
parameters = {
    'eavl_metric' : 'logloss',
    'max_depth' : 5,
    'colsample_bytree' : 0.4,
    'subsample' : 0.8
}

#instantiating the model
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', parameters=parameters, num_boost_round=10)

# TRAIN MODEL
model = xgb_clf.fit(X_train, y_train)

#FEATURE IMPORTANCE - GRAPHICAL
xgb.plot_importance(model)

[20:52:19] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [40]:
#predicting on the testing data
y_pred = xgb_clf.predict(data_test).astype('int')

#setting a threshold.
y_pred = (xgb_clf.predict_proba(data_test)[:, 1] >= 0.21).astype('int')
y_pred[0:10]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [41]:
#saving the prediction as a new column in data_test
data_test['prediction'] = y_pred
data_test.head()

uxp_times_bought  uxp_reordered_ratio  u_num_of_orders  \
user_id product_id                                                           
3       248                        1             0.000000               12   
        1005                       1             0.000000               12   
        1819                       3             0.666667               12   
        7503                       1             0.000000               12   
        8021                       1             0.000000               12   

                    u_avg_prd  dow_most_orders_u  hod_most_orders_u  \
user_id product_id                                                    
3       248          7.333333                  0                 16   
        1005         7.333333                  0                 16   
        1819         7.333333                  0                 16   
        7503         7.333333                  0                 16   
        8021         7.333333                  0                 16   

                    prd_count_p  p_reordered_ratio  aisle_id  department_id  \
user_id product_id                                                            
3       248                6371           0.400251       117             19   
        1005                463           0.440605        94              7   
        1819               2424           0.492162        88             13   
        7503              12474           0.553551       117             19   
        8021              27864           0.591157        54             17   

                    prediction  
user_id product_id              
3       248                  0  
        1005                 0  
        1819                 0  
        7503                 0  
        8021                 0

In [42]:
# Reset the index
final = data_test.reset_index()
# Keep only the required columns to create our submission file (for chapter 6)
final = final[['product_id', 'user_id', 'prediction']]

gc.collect()
final.head()

,product_id,user_id,prediction
0,248,3,0
1,1005,3,0
2,1819,3,0
3,7503,3,0
4,8021,3,0


In [45]:
#Creating a submission file
orders = pd.read_csv('orders.csv')
orders_test = orders.loc[orders.eval_set == 'test', ['user_id', 'order_id']]
orders_test.head()

,user_id,order_id
38,3,2774568
44,4,329954
53,6,1528013
96,11,1376945
102,12,1356845


In [46]:
#merging our prediction with orders_test
final = final.merge(orders_test, on='user_id', how='left')
final.head()

,product_id,user_id,prediction,order_id
0,248,3,0,2774568
1,1005,3,0,2774568
2,1819,3,0,2774568
3,7503,3,0,2774568
4,8021,3,0,2774568


In [47]:
#remove user_id column
final = final.drop('user_id', axis=1)

In [48]:
#convert product_id as integer
final['product_id'] = final.product_id.astype(int)

## Remove all unnecessary objects
del orders
del orders_test
gc.collect()

final.head()

,product_id,prediction,order_id
0,248,0,2774568
1,1005,0,2774568
2,1819,0,2774568
3,7503,0,2774568
4,8021,0,2774568


In [56]:
d = dict()
for row in final.itertuples():
    if row.prediction== 1:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in final.order_id:
    if order not in d:
        d[order] = 'None'
        
gc.collect()

#We now check how the dictionary were populated (open hidden output)
#d

46

In [57]:
#Convert the dictionary into a DataFrame
sub = pd.DataFrame.from_dict(d, orient='index')

#Reset index
sub.reset_index(inplace=True)
#Set column names
sub.columns = ['order_id', 'products']

sub.head()

,order_id,products
0,2774568,9387 17668 21903 22035 39190 43961 47766
1,1528013,21903 38293
2,1376945,8309 8670 14947 27959 28465 33572 34658 35640 ...
3,1356845,7076 10863 13176 14992 21616 28134
4,2161313,196 10441 11266 12427 14715 27839 37710


In [58]:
sub.to_csv('sub.csv', index=False)

In [59]:
rst2 = pd.read_csv('sub.csv')
rst2

,order_id,products
0,2774568,9387 17668 21903 22035 39190 43961 47766
1,1528013,21903 38293
2,1376945,8309 8670 14947 27959 28465 33572 34658 35640 ...
3,1356845,7076 10863 13176 14992 21616 28134
4,2161313,196 10441 11266 12427 14715 27839 37710
5,1416320,5134 17948 21137 21903 24852 28985 41950
6,1735923,196 2192 5183 12108 14252 15131 17008 18174 24...
7,1980631,6184 9387 13575 13914 22362 41400 46061
8,139655,13176 21903 22935 24964 27845
9,1411408,22008 43758
